In [1]:
# Import libraries
import os
import pandas as pd
import sqlite3

In [2]:
# Set working directory
desired_dir = 'C:/Users/idris/Downloads/Datasets'
os.chdir(desired_dir)

In [3]:
# Read SAS files from https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Laboratory&Cycle=2017-2020
ghb = pd.read_sas('P_GHB.XPT')
fast_gluc = pd.read_sas('P_GLU.XPT')

In [4]:
# Establish a connection to SQLite database
conn = sqlite3.connect('Lab Records.db')

In [5]:
# Move tables to Lab Records database
ghb.to_sql('aic',conn, if_exists='replace',index=False)
fast_gluc.to_sql('fast_gluc',conn, if_exists='replace',index=False)

5090

In [6]:
# Full join the fasting glucose and AIC tableS
lab_records = '''
CREATE TABLE 
    lab_records AS 
SELECT 
    f.SEQN,
    f.LBXGLU,
    a.LBXGH 
FROM 
    fast_gluc f
FULL JOIN 
    aic a ON f.SEQN = a.SEQN
WHERE 
    f.SEQN IS NOT NULL AND a.SEQN IS NOT NULL
GROUP BY 
    f.SEQN;
'''
# Execute query with database cursor 
cursor = conn.cursor()
cursor.execute(lab_records)
cursor.execute('SELECT * FROM lab_records')

# Return first 50 rows
rows = cursor.fetchmany(50)
for row in rows:
    print(row)

(109264.0, 97.0, 5.3)
(109271.0, 103.0, 5.6)
(109274.0, 154.0, 5.7)
(109277.0, 92.0, 5.3)
(109282.0, 95.0, 5.5)
(109286.0, 92.0, 5.7)
(109290.0, 106.0, 8.4)
(109292.0, 181.0, 6.4)
(109297.0, 92.0, 5.1)
(109300.0, 103.0, 5.4)
(109305.0, 105.0, 5.2)
(109307.0, 103.0, 5.3)
(109313.0, 97.0, 5.9)
(109317.0, 88.0, 5.0)
(109322.0, 89.0, 5.4)
(109323.0, 104.0, 5.2)
(109324.0, 86.0, None)
(109326.0, 95.0, 5.1)
(109327.0, 98.0, 5.6)
(109330.0, 109.0, 5.6)
(109331.0, 96.0, 5.3)
(109332.0, 88.0, 5.0)
(109335.0, 98.0, 6.3)
(109340.0, 110.0, 5.8)
(109342.0, 97.0, 5.2)
(109346.0, None, None)
(109350.0, None, 5.3)
(109353.0, None, None)
(109358.0, None, None)
(109360.0, 97.0, 4.6)
(109365.0, 100.0, 5.3)
(109367.0, 96.0, 5.7)
(109368.0, None, None)
(109369.0, 103.0, 5.6)
(109371.0, 102.0, 5.8)
(109376.0, 99.0, 5.4)
(109377.0, 100.0, 5.4)
(109378.0, 97.0, 5.3)
(109379.0, 106.0, 5.5)
(109380.0, 89.0, 5.2)
(109382.0, 129.0, 6.5)
(109392.0, 88.0, 5.3)
(109394.0, 97.0, 6.0)
(109398.0, 95.0, 5.3)
(109399.0, 

In [7]:
# Create new columns for diabetic status and doctor follow-up based on CDC guidelines 
diabetic_followup = '''
CREATE TABLE 
    diabetes AS 
SELECT 
    SEQN, 
    LBXGLU, 
    LBXGH,
    CASE
        WHEN LBXGLU < 100 AND LBXGH < 5.7 THEN 'Non-Diabetic' 
        WHEN (LBXGLU >= 100 AND LBXGLU < 126) OR (LBXGH >= 5.7 AND LBXGH < 6.5) THEN 'Pre-Diabetic'
        WHEN (LBXGLU >= 100 AND LBXGLU < 126 AND LBXGH IS NULL) OR 
             (LBXGLU IS NULL AND LBXGH >= 5.7 AND LBXGH < 6.5) THEN 'Pre-Diabetic'
        WHEN (LBXGLU >= 126 OR LBXGH >= 6.5) THEN 'Diabetic'
        WHEN (LBXGLU >= 126 AND LBXGH IS NULL) OR 
             (LBXGLU IS NULL AND LBXGH >= 6.5) THEN 'Diabetic'
        ELSE 'Inconclusive'
    END AS Diabetic_Status,
    CASE
        WHEN LBXGLU < 100 AND LBXGH < 5.7 THEN 'No'
        ELSE 'Yes'
    END AS Doctor_Followup
FROM 
    lab_records;
'''
cursor = conn.cursor()
cursor.execute(diabetic_followup)
cursor.execute('SELECT * FROM diabetes')

rows = cursor.fetchmany(50)
for row in rows:
    print(row)

(109264.0, 97.0, 5.3, 'Non-Diabetic', 'No')
(109271.0, 103.0, 5.6, 'Pre-Diabetic', 'Yes')
(109274.0, 154.0, 5.7, 'Pre-Diabetic', 'Yes')
(109277.0, 92.0, 5.3, 'Non-Diabetic', 'No')
(109282.0, 95.0, 5.5, 'Non-Diabetic', 'No')
(109286.0, 92.0, 5.7, 'Pre-Diabetic', 'Yes')
(109290.0, 106.0, 8.4, 'Pre-Diabetic', 'Yes')
(109292.0, 181.0, 6.4, 'Pre-Diabetic', 'Yes')
(109297.0, 92.0, 5.1, 'Non-Diabetic', 'No')
(109300.0, 103.0, 5.4, 'Pre-Diabetic', 'Yes')
(109305.0, 105.0, 5.2, 'Pre-Diabetic', 'Yes')
(109307.0, 103.0, 5.3, 'Pre-Diabetic', 'Yes')
(109313.0, 97.0, 5.9, 'Pre-Diabetic', 'Yes')
(109317.0, 88.0, 5.0, 'Non-Diabetic', 'No')
(109322.0, 89.0, 5.4, 'Non-Diabetic', 'No')
(109323.0, 104.0, 5.2, 'Pre-Diabetic', 'Yes')
(109324.0, 86.0, None, 'Inconclusive', 'Yes')
(109326.0, 95.0, 5.1, 'Non-Diabetic', 'No')
(109327.0, 98.0, 5.6, 'Non-Diabetic', 'No')
(109330.0, 109.0, 5.6, 'Pre-Diabetic', 'Yes')
(109331.0, 96.0, 5.3, 'Non-Diabetic', 'No')
(109332.0, 88.0, 5.0, 'Non-Diabetic', 'No')
(109335.0,

In [8]:
# Filter for patients who need to be tested for diabetes
inconclusive = '''
SELECT 
    SEQN
FROM
    diabetes 
WHERE Diabetic_Status = 'Inconclusive';
'''
cursor = conn.cursor()
cursor.execute(inconclusive)

rows = cursor.fetchmany(50)
for row in rows:
    print(row)

(109324.0,)
(109346.0,)
(109350.0,)
(109353.0,)
(109358.0,)
(109368.0,)
(109437.0,)
(109490.0,)
(109523.0,)
(109726.0,)
(109743.0,)
(109823.0,)
(109972.0,)
(110034.0,)
(110048.0,)
(110121.0,)
(110181.0,)
(110201.0,)
(110222.0,)
(110254.0,)
(110282.0,)
(110302.0,)
(110401.0,)
(110468.0,)
(110490.0,)
(110535.0,)
(110565.0,)
(110586.0,)
(110650.0,)
(110661.0,)
(110757.0,)
(110776.0,)
(110842.0,)
(110902.0,)
(110980.0,)
(110996.0,)
(111038.0,)
(111044.0,)
(111051.0,)
(111135.0,)
(111399.0,)
(111414.0,)
(111490.0,)
(111505.0,)
(111549.0,)
(111563.0,)
(111596.0,)
(111651.0,)
(111667.0,)
(111687.0,)


In [9]:
# Filter for pre-diabetic and diabetic patients to follow up with
diabetic_patients = '''
SELECT 
    SEQN
FROM
    diabetes 
WHERE Diabetic_Status = 'Diabetic' OR 'Pre-Diabetic';
'''
cursor = conn.cursor()
cursor.execute(diabetic_patients)

rows = cursor.fetchmany(50)
for row in rows:
    print(row)


(109382.0,)
(109415.0,)
(109480.0,)
(109552.0,)
(109558.0,)
(109563.0,)
(109565.0,)
(109610.0,)
(109612.0,)
(109620.0,)
(109721.0,)
(109725.0,)
(109742.0,)
(109803.0,)
(109841.0,)
(109843.0,)
(109844.0,)
(109881.0,)
(109941.0,)
(109999.0,)
(110005.0,)
(110057.0,)
(110064.0,)
(110124.0,)
(110151.0,)
(110185.0,)
(110188.0,)
(110207.0,)
(110292.0,)
(110304.0,)
(110353.0,)
(110390.0,)
(110410.0,)
(110417.0,)
(110418.0,)
(110434.0,)
(110488.0,)
(110532.0,)
(110538.0,)
(110578.0,)
(110608.0,)
(110651.0,)
(110656.0,)
(110702.0,)
(110737.0,)
(110762.0,)
(110778.0,)
(110822.0,)
(110834.0,)
(110870.0,)


In [10]:
# Pull breakdown of diabetes status 
status = '''
SELECT 
    Diabetic_Status, 
    COUNT(*) AS Total 
FROM 
    diabetes
GROUP BY 
    diabetic_Status;
'''
cursor = conn.cursor()
cursor.execute(status)

rows = cursor.fetchall()
for row in rows:
    print(row)

('Diabetic', 529)
('Inconclusive', 337)
('Non-Diabetic', 1636)
('Pre-Diabetic', 2588)


In [11]:
# Pull breakdown of followups 
followup = '''
SELECT 
    Doctor_Followup, 
    COUNT(*) AS Total
FROM
    diabetes
GROUP BY
    Doctor_Followup;
'''
cursor = conn.cursor()
cursor.execute(followup)

rows = cursor.fetchall()
for row in rows:
    print(row)

('No', 1636)
('Yes', 3454)


In [12]:
# SQL query to retrieve data from the 'diabetes' table
query = "SELECT * FROM diabetes;"

# Fetch data from SQL into a pandas DataFrame
diabetes_data = pd.read_sql_query(query, conn)

# Export the pandas DataFrame to a CSV file
diabetes_data.to_csv('diabetes_data.csv', index=False)  # Change 'diabetes_data.csv' to your desired file name

In [13]:
# Commit changes and close the connection
conn.commit()
conn.close()